In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
url = "https://mangalib.me/manga-list?types[]=1"
response = requests.get(url)

In [3]:
response.text

'<!doctype html>\n<html id="site_type" lang="ru" data-id="1">\n  <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0">\n    <meta charset="utf-8"/>\n    <meta http-equiv="Pragma" content="no-cache"/>\n    <meta http-equiv="Expires" content="-1"/>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/>\n    <title>Каталог манги. Читать мангу онлайн.\n</title>\n    <meta name="description" content="Огромное количество самой разной манги, удобный поиск и сортировка. Жанры и категории. огромный каталог манги\n"/>\n    <meta name="keywords" content="манга, читать мангу, манга яой, манга онлайн, manga, read, mangaread, манга на русском, перевод, манги, на русский, скачать мангу, онлайн, манга про любовь, романтика, для девочек, читать, манги, интересная манга, русская манга, манга любовь, токийский гуль, манга для взрослых, яой манга, манга читать онлайн, читать мангу онлайн\n"/>\n\n    <meta name="mobile-web-app-capable" content="

In [4]:
from bs4 import BeautifulSoup


In [5]:
bs = BeautifulSoup(response.text)
bs

<!DOCTYPE html>

<html data-id="1" id="site_type" lang="ru">
<head>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta charset="utf-8">
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Каталог манги. Читать мангу онлайн.
</title>
<meta content="Огромное количество самой разной манги, удобный поиск и сортировка. Жанры и категории. огромный каталог манги
" name="description"/>
<meta content="манга, читать мангу, манга яой, манга онлайн, manga, read, mangaread, манга на русском, перевод, манги, на русский, скачать мангу, онлайн, манга про любовь, романтика, для девочек, читать, манги, интересная манга, русская манга, манга любовь, токийский гуль, манга для взрослых, яой манга, манга читать онлайн, читать мангу онлайн
" name="keywords"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-